In [71]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import OneHotEncoder
import joblib

# Criacao da base de dados

In [73]:
def monta_base(sequencias, labels, define_encoder = False):
    base_seqs = pd.read_csv(sequencias)
    base_labels = pd.read_csv(labels)

    base_seqs["resname"] = base_seqs["sequence"].apply(lambda x: list(x))
    base_seqs = base_seqs[["target_id", "sequence", "resname"]].explode("resname", ignore_index=True)
    base_seqs["resid"] = base_seqs.groupby("target_id").cumcount() + 1
    base_seqs["target_num"] = base_seqs["target_id"] + "_" + base_seqs["resid"].astype(str)
    
    base_set = pd.merge(base_seqs[["target_num", "sequence", 'target_id']], base_labels, left_on="target_num", right_on="ID").drop("target_num", axis=1)


    base_set = base_set.dropna(how='any', axis=0).reset_index().drop('index', axis=1)

    base_set['first'] = (base_set['ID'] == base_set['ID'].iloc[0]).astype(int)
    base_set['last'] = (base_set['ID'] == base_set['ID'].iloc[-1]).astype(int)
    base_set['lead1'] = base_set['resname'].shift(-1)
    base_set['lead2'] = base_set['resname'].shift(-2)
    base_set['lead3'] = base_set['resname'].shift(-3)
    base_set['lag1'] = base_set['resname'].shift(1)
    base_set['lag2'] = base_set['resname'].shift(2)
    base_set['lag3'] = base_set['resname'].shift(3)

    colunas_one_hot = ['resname', 'lead1', 'lead2', 'lead3', 'lag1', 'lag2', 'lag3']
    df_one_hot = base_set[colunas_one_hot]
    df_one_hot['ID'] = '0'
    colunas_one_hot.append('ID')

    if define_encoder:
        enc = OneHotEncoder(handle_unknown='ignore')
        codificados = enc.fit(df_one_hot)
        joblib.dump(enc, "onehot_encoder.pkl")

        
    enc = joblib.load("onehot_encoder.pkl")
    codificados = enc.transform(df_one_hot).toarray()

    renames = []
    for coluna, cats in zip(colunas_one_hot, enc.categories_):
        # print(coluna, cat)
        for cat in cats:
            if cat is None:
                cat = 'None'
            renames.append(coluna + '_' + cat)

    codificados = pd.DataFrame(codificados, columns=renames).iloc[:, :-1]

    base_set = pd.concat([base_set, codificados], axis=1)

    cols = renames[:-1]
    cols.extend(['first', 'last'])

    X = base_set[cols].fillna(99)
    y = base_set[['x_1', 'y_1', 'z_1']]

    return X, y

In [75]:
X_train, y_train = monta_base("train_sequences.csv", "train_labels.csv", False)
X_validation, y_validation = monta_base("validation_sequences.csv", "validation_labels.csv", False)

C:\Users\pedro\AppData\Local\Temp\ipykernel_32028\1918933705.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_one_hot['ID'] = '0'
C:\Users\pedro\AppData\Local\Temp\ipykernel_32028\1918933705.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_one_hot['ID'] = '0'


# Rede Neural

In [76]:
X_validation

,resname_-,resname_A,resname_C,resname_G,resname_U,lead1_-,lead1_A,lead1_C,lead1_G,lead1_U,...,lag2_U,lag2_None,lag3_-,lag3_A,lag3_C,lag3_G,lag3_U,lag3_None,first,last
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0
2511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0
2512,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0
2513,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0


In [77]:
X_train

,resname_-,resname_A,resname_C,resname_G,resname_U,lead1_-,lead1_A,lead1_C,lead1_G,lead1_U,...,lag2_U,lag2_None,lag3_-,lag3_A,lag3_C,lag3_G,lag3_U,lag3_None,first,last
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130945,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0
130946,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0
130947,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0
130948,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0


In [81]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.metrics import f1_score, roc_auc_score
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Supondo que df seja um DataFrame Pandas já carregado

# Convertendo para tensores
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)  # Agora float, pois são valores binários
Y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).to(device)

X_validation_tensor = torch.tensor(X_validation.values, dtype=torch.float32).to(device)  # Agora float, pois são valores binários
Y_validation_tensor = torch.tensor(y_validation.values, dtype=torch.float32).to(device)

# Criando um Dataset personalizado
class RNA3DDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

# Criando instância do Dataset
dataset_train = RNA3DDataset(X_train_tensor, Y_train_tensor)
dataset_val = RNA3DDataset(X_validation_tensor, Y_validation_tensor)

# Criando DataLoader para carregar os dados em batches
BATCH_SIZE = 32
# train_loader = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
# val_loader = DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=True)
# Definições
INPUT_DIM = X_train_tensor.shape[1]  # 43 valores binários
LSTM_UNITS = 64
OUTPUT_DIM = 3  # Coordenadas X, Y, Z por nucleotídeo

# Criando o modelo
class RNA3DModel(nn.Module):
    def __init__(self):
        super(RNA3DModel, self).__init__()
        self.lstm = nn.LSTM(INPUT_DIM, LSTM_UNITS, batch_first=True)
        self.fc = nn.Linear(LSTM_UNITS, OUTPUT_DIM)
    
    def forward(self, x):
        x = x.unsqueeze(1)
        x, _ = self.lstm(x)  # Pegamos todas as saídas da LSTM
        x = self.fc(x[:, -1, :])  # Pegamos apenas a última saída da sequência
        return x
    
# Instanciando o modelo
model = RNA3DModel().to(device)

# Função de perda e otimizador
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Função de treinamento
def train_model(model, criterion, optimizer, dataloaders, dataset_sizes, scheduler=None, num_epochs=25, patience=15):
    best_loss = float('inf')
    epochs_no_improve = 0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)
        
        for phase in ['train', 'val']:
            is_train = phase == 'train'
            model.train() if is_train else model.eval()
            
            running_loss = 0.0
            progress_bar = tqdm(dataloaders[phase], desc=f'{phase.capitalize()}')
            
            for batch_X, batch_Y in progress_bar:
                batch_X = batch_X.to(device, dtype=torch.float32)
                batch_Y = batch_Y.to(device, dtype=torch.float32)
                with torch.set_grad_enabled(is_train):
                    output = model(batch_X)
                    loss = criterion(output, batch_Y)
                    
                    if is_train:
                        optimizer.zero_grad()
                        loss.backward()
                        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                        optimizer.step()
                    
                running_loss += loss.item() * batch_X.size(0)
                current_loss = running_loss / dataset_sizes[phase]
                progress_bar.set_postfix({'loss': current_loss})
            
            epoch_loss = running_loss / dataset_sizes[phase]
            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f}')
            
            if phase == 'val' and scheduler:
                scheduler.step(epoch_loss)
            
            if phase == 'val':
                if epoch_loss < best_loss:
                    best_loss = epoch_loss
                    epochs_no_improve = 0
                    torch.save(model.state_dict(), 'best_model.pt')
                else:
                    epochs_no_improve += 1
        
        if epochs_no_improve >= patience:
            print(f'Early stopping! No improvement in validation loss for {patience} epochs.')
            print(f'Best Loss: {best_loss}')
            break
    
    return model

# Criando DataLoaders e scheduler
batch_size = 32
fine_dataloaders = {
    'train': DataLoader(dataset_train, batch_size=batch_size, shuffle=True),
    'val': DataLoader(dataset_val, batch_size=batch_size, shuffle=True),
}

fine_dataset_sizes = {
    'train': len(dataset_train),
    'val': len(dataset_val)
}

l2_lambda = 0.3
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)#, weight_decay=l2_lambda)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=30, verbose=True)

# Treinando o modelo
model = train_model(model, criterion, optimizer, fine_dataloaders, fine_dataset_sizes, scheduler=scheduler, num_epochs=500, patience=50)


Epoch 0/499
----------


Train: 100%|██████████| 4093/4093 [00:08<00:00, 467.36it/s, loss=2.26e+4]


Train Loss: 22575.8171


Val: 100%|██████████| 79/79 [00:00<00:00, 959.81it/s, loss=5.96e+33]


Val Loss: 5964214483242442418769038774632448.0000
Epoch 1/499
----------


...

Val Loss: 5964214483242442418769038774632448.0000
Epoch 65/499
----------


Train: 100%|██████████| 4093/4093 [00:08<00:00, 493.92it/s, loss=1.62e+4]


Train Loss: 16248.5749


Val: 100%|██████████| 79/79 [00:00<00:00, 861.30it/s, loss=5.96e+33]

Val Loss: 5964214483242442418769038774632448.0000
Early stopping! No improvement in validation loss for 50 epochs.
Best Loss: 5.964214471429098e+33


In [66]:
for batch_X, batch_Y in DataLoader(dataset_val, batch_size=batch_size, shuffle=True):
    print(batch_X.shape)  # Deve ser (batch_size, input_dim)
    print(batch_Y.shape)
    

torch.Size([16, 36])
torch.Size([16, 3])
torch.Size([16, 36])
torch.Size([16, 3])
torch.Size([16, 36])
torch.Size([16, 3])
torch.Size([16, 36])
torch.Size([16, 3])
torch.Size([16, 36])
torch.Size([16, 3])
torch.Size([16, 36])
torch.Size([16, 3])
torch.Size([16, 36])
torch.Size([16, 3])
torch.Size([16, 36])
torch.Size([16, 3])
torch.Size([16, 36])
torch.Size([16, 3])
torch.Size([16, 36])
torch.Size([16, 3])
torch.Size([16, 36])
torch.Size([16, 3])
torch.Size([16, 36])
torch.Size([16, 3])
torch.Size([16, 36])
torch.Size([16, 3])
torch.Size([16, 36])
torch.Size([16, 3])
torch.Size([16, 36])
torch.Size([16, 3])
torch.Size([16, 36])
torch.Size([16, 3])
torch.Size([16, 36])
torch.Size([16, 3])
torch.Size([16, 36])
torch.Size([16, 3])
torch.Size([16, 36])
torch.Size([16, 3])
torch.Size([16, 36])
torch.Size([16, 3])
torch.Size([16, 36])
torch.Size([16, 3])
torch.Size([16, 36])
torch.Size([16, 3])
torch.Size([16, 36])
torch.Size([16, 3])
torch.Size([16, 36])
torch.Size([16, 3])
torch.Size([16, 